# AutoGraph应用

In [1]:
import os

import tensorflow as tf
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
tf.__version__

'2.1.0'

## tf.Module基类

前面在介绍Autograph的编码规范时提到构建Autograph时应该避免在@tf.function修饰的函数内部定义tf.Variable；

但是如果在函数外部定义tf.Variable的话，又会显得这个函数有外部变量依赖，封装不够完美。

一种简单的思路是定义一个类，并将相关的tf.Variable创建放在类的初始化方法中。而将函数的逻辑放在其他方法中。

在TensorFlow中提供了一个基类tf.Module，通过继承它构建子类，有如下优势：
- 可以非常方便地管理变量；
- 可以非常方便地管理它引用的其它Module；
- 能够利用tf.saved_model保存模型，并实现跨平台部署使用；

实际上，tf.keras.models.Model,tf.keras.layers.Layer 都是继承自tf.Module的，提供了方便的变量管理和所引用的子模块管理的功能。

In [2]:
x = tf.Variable(1.0, dtype=tf.float32)

#在tf.function中用input_signature限定输入张量的签名类型：shape和dtype
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype = tf.float32)])    
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)

In [3]:
add_print(tf.constant(3.0))
# add_print(tf.constant(3)) # 输入 不符合 【输入张量定义签名】的参数将报错

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [4]:
# 基于tf.Module实现
class DemoModule(tf.Module):
    def __init__(self, init_value=tf.constant(0.0), name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:  #相当于with tf.name_scope("demo_module")
            self.x = tf.Variable(init_value, dtype=tf.float32, trainable=True)


    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])  
    def addprint(self, a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return(self.x)

In [5]:
#执行
demo = DemoModule(init_value=tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [6]:
#查看模块中的全部变量和全部可训练变量
print(demo.variables)
print(demo.trainable_variables)

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [7]:
#查看模块中的全部子模块
demo.submodules

()

In [8]:
# 使用tf.saved_model 保存模型，并指定需要跨平台部署的方法
path_save = "/home/sq/data/model/tf2/demo/1"
tf.saved_model.save(demo, path_save, signatures={"serving_default": demo.addprint})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/sq/data/model/tf2/demo/1/assets


In [9]:
#加载模型
demo2 = tf.saved_model.load(path_save)
demo2.addprint(tf.constant(5.0))

11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [10]:
# 查看模型文件相关信息，红框标出来的输出信息在模型部署和跨平台使用时有可能会用到
!saved_model_cli show --dir path_save --all

/usr/bin/sh: saved_model_cli: 未找到命令


In [11]:
import datetime

# 创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "/home/sq/data/model/tf2/demo/log"
logdir_stamp = f'{logdir}/{stamp}'
writer = tf.summary.create_file_writer(logdir_stamp)

#开启autograph跟踪
tf.summary.trace_on(graph=True, profiler=True) 

#执行autograph
demo = DemoModule(init_value = tf.constant(0.0))
result = demo.addprint(tf.constant(5.0))

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name="demomodule",
        step=0,
        profiler_outdir=logdir_stamp)

5


In [12]:
#启动 tensorboard在jupyter中的魔法命令
%reload_ext tensorboard

from tensorboard import notebook
notebook.list()


No known TensorBoard instances running.


In [13]:
notebook.start(f"--logdir {logdir}")

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.